In [1]:
import wandb
wandb.login(key = "eb9574fa5b11da36782604ea27df8bf1989ddefd")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: megh_m (megh_m-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
import os
os.environ["WANDB_SILENT"] = "false"
os.environ["WANDB_START_METHOD"] = "thread"
os.environ["WANDB_API_KEY"] = "eb9574fa5b11da36782604ea27df8bf1989ddefd"

In [3]:
import numpy as np
import torch
import torch.nn as nn

In [4]:
class CharEmbed(nn.Module):
    def __init__(self, input_dim, embed_dim):
        super(CharEmbed, self).__init__()
        self.embed = nn.Embedding(input_dim, embed_dim)
    
    def forward(self, input_seq):
        return self.embed(input_seq)


In [5]:
class EncoderRNN(nn.Module):
    def __init__(self, input_dim, embed_dim, hidden_dim, n_layers=1, 
                 cell_type='GRU', dropout=0.0, bidirectional=False):
        super(EncoderRNN, self).__init__()
        self.embed = nn.Embedding(input_dim, embedding_dim)
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.cell_type = cell_type
        self.bidirectional = bidirectional #to allow forward and backward time step data processing
        # Cell type options GRU, LSTM & vanilla RNN
        if cell_type == 'GRU':
            self.rnn = nn.GRU(embed_dim, hidden_dim, n_layers, dropout=dropout if n_layers > 1 else 0, bidirectional=bidirectional)
        elif cell_type == 'LSTM':
            self.rnn = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout=dropout if n_layers > 1 else 0, bidirectional=bidirectional)
        else: 
            self.rnn = nn.RNN(embed_dim, hidden_dim, n_layers, dropout=dropout if n_layers > 1 else 0, bidirectional=bidirectional)
    
    def forward(self, input_seq, input_lengths, hidden=None):
        # Convert word indices to embeddings
        embedded = self.embed(input_seq)
        
        # Pack padded batch of sequences for RNN module
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        
        # Forward pass through RNN
        outputs, hidden = self.rnn(packed, hidden)
        
        # Unpack padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        
        # If bidirectional, sum bidirectional outputs, ie backward and forward
        if self.bidirectional:
            outputs = outputs[:, :, :self.hidden_dim] + outputs[:, :, self.hidden_dim:]
        
        return outputs, hidden

class DecoderRNN(nn.Module): #Basically similar to the encoder, will have a softmax to predict next char
    def __init__(self, output_dim, embed_dim, hidden_dim, n_layers=1, 
                 cell_type='GRU', dropout=0.0):
        super(DecoderRNN, self).__init__()
        self.embed = nn.Embedding(output_dim, embed_dim)
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.n_layers = n_layers
        self.cell_type = cell_type
        
        if cell_type == 'GRU':
            self.rnn = nn.GRU(embed_dim, hidden_dim, n_layers, dropout=dropout if n_layers > 1 else 0)
        elif cell_type == 'LSTM':
            self.rnn = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout=dropout if n_layers > 1 else 0)
        else:
            self.rnn = nn.RNN(embed_dim, hidden_dim, n_layers, dropout=dropout if n_layers > 1 else 0)
        
        self.out = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input, hidden):
        # Get embedding of current input character
        embedded = self.embed(input).unsqueeze(0)
        
        # Forward pass through decoder
        output, hidden = self.rnn(embedded, hidden)
        
        # Predict next character probabilities
        output = self.softmax(self.out(output.squeeze(0)))
        
        return output, hidden
